# Quick Start in 30s

## Build the index

In [12]:
from leann.api import LeannBuilder

builder = LeannBuilder(backend_name="hnsw")
builder.add_text("C# is a powerful programming language")
builder.add_text("Python is a powerful programming language and it is very popular")
builder.add_text("Machine learning transforms industries")
builder.add_text("Neural networks process complex data")
builder.add_text("Leann is a great storage saving engine for RAG on your MacBook")
builder.build_index("knowledge.leann")

INFO: Computing embeddings for 1 texts using SentenceTransformer, model: 'facebook/contriever-msmarco'
INFO: Using cached model: facebook/contriever-msmarco
INFO: Starting embedding computation...
INFO: Generated 1 embeddings, dimension: 768


Writing passages: 100%|██████████| 5/5 [00:00<00:00, 16345.69chunk/s]


INFO: Computing embeddings for 5 texts using SentenceTransformer, model: 'facebook/contriever-msmarco'
INFO: Using cached model: facebook/contriever-msmarco
INFO: Starting embedding computation...


Batches: 100%|██████████| 1/1 [00:00<00:00, 50.05it/s]
INFO:leann_backend_hnsw.hnsw_backend:INFO: Converting HNSW index to CSR-pruned format...


INFO: Generated 5 embeddings, dimension: 768
M: 64 for level: 0
Starting conversion: knowledge.index -> knowledge.csr.tmp
[0.00s] Reading Index HNSW header...
[0.00s]   Header read: d=768, ntotal=5
[0.00s] Reading HNSW struct vectors...
  Reading vector (dtype=<class 'numpy.float64'>, fmt='d')... Count=6, Bytes=48
[0.00s]   Read assign_probas (6)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=7, Bytes=28
[0.21s]   Read cum_nneighbor_per_level (7)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=5, Bytes=20
[0.33s]   Read levels (5)
[0.44s]   Probing for compact storage flag...
[0.44s]   Found compact flag: False
[0.44s]   Compact flag is False, reading original format...
[0.44s]   Probing for potential extra byte before non-compact offsets...
[0.44s]   Found and consumed an unexpected 0x00 byte.
  Reading vector (dtype=<class 'numpy.uint64'>, fmt='Q')... Count=6, Bytes=48
[0.44s]   Read offsets (6)
[0.54s]   Attempting to read neighbors vector...
  Rea

INFO:leann_backend_hnsw.hnsw_backend:✅ CSR conversion successful.
INFO:leann_backend_hnsw.hnsw_backend:INFO: Replaced original index with CSR-pruned version at 'knowledge.index'


## Search with real-time embeddings

In [13]:
from leann.api import LeannSearcher

searcher = LeannSearcher("knowledge.leann")
results = searcher.search("programming languages", top_k=2)
results

[read_HNSW - CSR NL v4] Reading metadata & CSR indices (manual offset)...
[read_HNSW NL v4] Read levels vector, size: 5
[read_HNSW NL v4] Reading Compact Storage format indices...
[read_HNSW NL v4] Read compact_level_ptr, size: 10
[read_HNSW NL v4] Read compact_node_offsets, size: 6
[read_HNSW NL v4] Read entry_point: 4, max_level: 0
[read_HNSW NL v4] Read storage fourcc: 0x6c6c756e
[read_HNSW NL v4 FIX] Detected FileIOReader. Neighbors size field offset: 326
[read_HNSW NL v4] Reading neighbors data into memory.
[read_HNSW NL v4] Read neighbors data, size: 20
[read_HNSW NL v4] Finished reading metadata and CSR indices.
INFO: Skipping external storage loading, since is_recompute is true.
INFO: Terminating server process (PID: 1311) for backend leann_backend_hnsw.hnsw_embedding_server...
INFO: Server process 1311 terminated.
🔍 DEBUG LeannSearcher.search() called:
  Query: 'programming languages'
  Top_k: 2
  Additional kwargs: {}
DEBUG: Found process on port 5557: /Users/yichuan/Desktop/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ Embedding server is ready!
[leann_backend_hnsw.hnsw_embedding_server LOG]: INFO: Registering backend 'diskann'
DEBUG: Found process on port 5560: /Users/yichuan/Desktop/code/LEANN/leann/.venv/bin/python -m leann_backend_hnsw.hnsw_embedding_server --zmq-port 5560 --model-name facebook/contriever-msmarco --passages-file knowledge.leann.meta.json[leann_backend_hnsw.hnsw_embedding_server LOG]: INFO: Registering backend 'hnsw'

DEBUG: model_matches: True, passages_matches: True, overall: True
✅ Existing server process (PID 5895) is compatible
[leann_backend_hnsw.hnsw_embedding_server LOG]: INFO:datasets:PyTorch version 2.7.1 available.
[leann_backend_hnsw.hnsw_embedding_server LOG]: INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: facebook/contriever-msmarco
[leann_backend_hnsw.hnsw_embedding_server LOG]: WARNING:sentence_transformers.SentenceTransformer:No sentence-transformers model found with name facebook/contriever-msmarco. Creating a new one with m

[SearchResult(id='0', score=np.float32(1.444752), text='C# is a powerful programming language', metadata={}),
 SearchResult(id='1', score=np.float32(1.394647), text='Python is a powerful programming language and it is very popular', metadata={})]

## Chat with LEANN using retrieved results

In [14]:
from leann.api import LeannChat

llm_config = {
    "type": "ollama",
    "model": "llama3.2:1b"
}

chat = LeannChat(index_path="knowledge.leann", llm_config=llm_config)
response = chat.ask(
    "Compare the two retrieved programming languages and say which one is more popular today.",
    top_k=2,
)
response

INFO:leann.chat:Attempting to create LLM of type='ollama' with model='llama3.2:1b'
INFO:leann.chat:Initializing OllamaChat with model='llama3.2:1b' and host='http://localhost:11434'


[read_HNSW - CSR NL v4] Reading metadata & CSR indices (manual offset)...
[read_HNSW NL v4] Read levels vector, size: 5
[read_HNSW NL v4] Reading Compact Storage format indices...
[read_HNSW NL v4] Read compact_level_ptr, size: 10
[read_HNSW NL v4] Read compact_node_offsets, size: 6
[read_HNSW NL v4] Read entry_point: 4, max_level: 0
[read_HNSW NL v4] Read storage fourcc: 0x6c6c756e
[read_HNSW NL v4 FIX] Detected FileIOReader. Neighbors size field offset: 326
[read_HNSW NL v4] Reading neighbors data into memory.
[read_HNSW NL v4] Read neighbors data, size: 20
[read_HNSW NL v4] Finished reading metadata and CSR indices.
INFO: Skipping external storage loading, since is_recompute is true.
🔍 DEBUG LeannSearcher.search() called:
  Query: 'Compare the two retrieved programming languages and say which one is more popular today.'
  Top_k: 2
  Additional kwargs: {}
DEBUG: Found process on port 5557: /Users/yichuan/Desktop/code/LEANN/leann/.venv/bin/python -m leann_backend_hnsw.hnsw_embedding_s

INFO:leann.chat:Sending request to Ollama: {'model': 'llama3.2:1b', 'prompt': 'Here is some retrieved context that might help answer your question:\n\nPython is a powerful programming language and it is very popular\n\nC# is a powerful programming language\n\nQuestion: Compare the two retrieved programming languages and say which one is more popular today.\n\nPlease provide the best answer you can based on this context and your knowledge.', 'stream': False, 'options': {}}


  Generated embedding shape: (1, 768)
  Embedding time: 0.09699082374572754 seconds
ZmqDistanceComputer initialized: d=768, metric=0
  Search time: 0.028768062591552734 seconds
  Backend returned: labels=2 results
  Processing 2 passage IDs:
    1. passage_id='1' -> SUCCESS: Python is a powerful programming language and it is very popular...
    2. passage_id='0' -> SUCCESS: C# is a powerful programming language...
  Final enriched results: 2 passages


"Based on my analysis, I would say that C# is currently more popular than Python. Here's why:\n\n1. **Industry adoption**: C# has been widely adopted in the .NET ecosystem, which includes Microsoft products like Visual Studio, Azure, and Office 365. This extensive industry support contributes to its higher popularity.\n2. **Microsoft's focus**: As a native Windows development language, C# is closely tied to Microsoft's ecosystem, making it more integrated with Windows-based systems. This integration has helped boost C#'s popularity among developers working on Windows-related projects.\n3. **Cross-platform compatibility**: While Python is gaining traction in other areas, such as data science and machine learning, C# remains particularly popular for developing Windows desktop applications, mobile apps, and games that run on both Windows and macOS/Android/iOS platforms.\n\nThat being said, Python's popularity has been growing rapidly due to its ease of use, flexibility, and extensive libr